In [1]:
'''
Book & Library Chatbot Assistant

Keeps track of your books, their ratings, and outputs recommendations based on NLP
'''

import nltk
import re
import pickle
import spacy
import pandas as pd
from datetime import datetime
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from collections import defaultdict

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\myaku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\myaku\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\myaku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# LOAD NLP FROM SPACY
#pip install spacy && python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

# LOAD IN THE DATAFRAME OF BOOKS
book_df = pd.read_csv("books.csv", sep=";", error_bad_lines=False, encoding="latin-1")

C:\Users\myaku\AppData\Local\Temp\ipykernel_12020\197791836.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_df = pd.read_csv("books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: e

In [3]:
'''
-----------------
CHECK IF ALL STRING ARE PRESENT IN TEXT
-----------------
'''
def check_all_present(string_list, text):
    for string in string_list:
        if string not in text.lower():
            return False
    return True
'''
-----------------
EXTRACT BOOK TITLE
-----------------
'''
def extract_title(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "WORK_OF_ART":
            return ent.text
    return None
'''
-----------------
EXTRACT RATING/REVIEW
-----------------
'''
def check_any_present(string_list, text):
    for string in string_list:
        if string.lower() in text.lower():
            return True
    return False

In [4]:
def interact_with_chatbot(assistant, user_id):
    print("Welcome to the Personal Book and Movie chatbot!")
    print("Type 'quit' to exit.")

    while True:
        user_input = input(f"{user_id}: ")
        if user_input.lower() == 'quit':
            break

        response = assistant.respond(user_id, user_input)
        print(f"Assistant: {response}")

    assistant.save_user_data()
    print("Goodbye!")

In [5]:
class UserModel:
    def __init__(self, name):
        self.name = name
        self.book_ratings = {}
        self.name_prompt = False
        self.add_book_prompt = False
        self.rating_prompt = False

In [6]:
class PersonalAssistant:
    '''
    -----------------
    CHATBOT INTIALIZATION
    -----------------
    '''
    def __init__(self, user_data_file):
        self.user_data_file = user_data_file
        self.user = self.load_user_data()
    '''
    -----------------
    USER DATA FUNCTIONS
    -----------------
    '''
    def load_user_data(self):
        try:
            with open(self.user_data_file, 'rb') as f:
                return pickle.load(f)
        except (FileNotFoundError, EOFError):
            return {}

    def save_user_data(self):
        with open(self.user_data_file, 'wb') as f:
            pickle.dump(self.user, f)

    def get_user_model(self, user_id):
        if user_id not in self.user:
            self.user[user_id] = UserModel("unknown")
        return self.user[user_id]
    
    '''
    -----------------
    CHAT RESPONSE HANDLING FUNCTIONS
    -----------------
    '''
    def respond(self, user_id, text):
        #LOAD USER DATA
        user_model = self.get_user_model(user_id)

        #CHANGE NAME - If flagged, use POS tagging or get input and set as the user's name
        if user_model.name_prompt:
            tagged_text = pos_tag(word_tokenize(text))
            first_name, last_name = None, None
            for i, (word, pos) in enumerate(tagged_text):
                if pos == 'NNP':
                    if not first_name:
                        first_name = word.capitalize()
                    else:
                        last_name = word.capitalize()
                        break
            if not first_name:
                user_model.name = text
            else:
                user_model.name = f"{first_name} {last_name}" if last_name else first_name
            user_model.name_prompt = False
            return f'Alright, i\'ve set your name as {user_model.name}'
        
        # NAME PROMPT IF DNE - If user doesn't have a name set the flag
        if user_model.name == 'unknown':
            user_model.name_prompt = True
            return f"Hello!, I've noticed I don't know your name in my knowledge base. Could you provide your first name so I know how to address you?"
        
        # BOOK PROMPT IF TITLE NONE - If NLP could not extract a title, then ask directly
        if user_model.add_book_prompt:
            try:
                title, rating = text.split(",")
                user_model.book_ratings[title.lower()] = rating
                user_model.add_book_prompt = False
                return f"Alright, I've added the book: {title} to your ratings with a score of {rating}"
            except:
                return f"Sorry, that follows in incorrect input format. Please enter the book title and rating separated by a comma."
        
        #CHANGE NAME - Change the user's name
        if check_all_present(['change', 'name'], text):
            user_model.name_prompt = True
            return f'Alright, let me know what you want to change your name to'
        
        #CHECK NAME - Print the user's name
        if check_all_present(['what', 'name'], text):
            return f"Your name is {user_model.name}"      
        
        #DETECT AND STORE RATINGS - When the user is giving their review for a book/rating.
        if re.search(r'rate.*?(\d+)', text):
            title = extract_title(text)
            rating = int(re.search(r'\d+', text).group())
            if title is not None:
                user_model.book_ratings[title.lower()] = rating
                return f"Alright, I've added {title.lower()} to your book ratings with a rating of {rating}"
            else:
                user_model.add_book_prompt = True
                return f"I'm having trouble determining the title of the book you're trying to add, could you give me the title of the book and rating split by a comma?"
            
            
        #PRINT RATINGS FOR CATEGORY
        if "ratings" in text.lower():
            category = re.search(r'ratings for (\w+)', text, re.IGNORECASE)
            if category:
                if category.group(1).lower() == 'books':
                    items = user_model.book_ratings
                elif category.group(1).lower() == 'movies':
                    items = user_model.movie_ratings
                response = f"{user_model.name}, your ratings for {category.group(1).lower()} are:\n"
                for item, rating in items.items():
                    response += f"- {item}: {rating}/10\n"
                return response
            else:
                return f"I couldn't find any ratings for the specified category, {user_model.name}."

        #HELLO RESPONSE
        if any(user_input in text for user_input in ['hello', 'hi', 'greetings']):
            return f"Hello, {user_model.name}! How can I help you today?" 
        
        #DOCUMENTATION
        if check_any_present(['documentation', 'docs', 'help', 'tutorial'], text):
            return f"Here's a list of things I can help you with:"
        
        #CONFUSED RESPONSE
        return f"I'm not sure what you're asking, {user_model.name}. Could you please rephrase?\nIf you need documentation let me know"

In [ ]:
user_id = "Matthew Y"
user_data_file = user_id + '.pkl'
assistant = PersonalAssistant(user_data_file)

interact_with_chatbot(assistant, user_id)

Welcome to the Personal Book and Movie chatbot!
Type 'quit' to exit.
Matthew Y: hi
Assistant: Hello!, I've noticed I don't know your name in my knowledge base. Could you provide your first name so I know how to address you?
Matthew Y: my name is Matthew
Assistant: Alright, i've set your name as Matthew
Matthew Y: I would like to rate the name of the wind as 5
Assistant: I'm having trouble determining the title of the book you're trying to add, could you give me the title of the book and rating split by a comma?
Matthew Y: Name of the wind
Assistant: Sorry, that follows in incorrect input format. Please enter the book title and rating separated by a comma.
Matthew Y: Name of the wind
Assistant: Sorry, that follows in incorrect input format. Please enter the book title and rating separated by a comma.
Matthew Y: Name of the wind, 5
Assistant: Alright, I've added the book: Name of the wind to your ratings with a score of  5
Matthew Y: ratings
Assistant: I couldn't find any ratings for the